In [1]:
import importlib
import numpy as np
import os
import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
% matplotlib inline
import tensorflow as tf
from keras import backend as K
import pickle
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import random
import cv2 as cv
import shutil
import scipy.signal as signal
import scipy

import preprocess.filesProc as filesProc
import preprocess.pseudoEcg as pseudoEcg
import preprocess.ecg as Ecg
import dataProc
import opdata
import analysis
import utils
import resUnet3d
import visualize

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
elec_pos = np.zeros((25, 2), np.float32)
x_ = np.linspace(93/10, 93-93/10, 5)
for i in range(5):
    elec_pos[i*5:(i+1)*5, 0] = x_
    elec_pos[i*5:(i+1)*5, 1] = np.flip(x_, 0)[i]
print(elec_pos)

[[ 9.3 83.7]
 [27.9 83.7]
 [46.5 83.7]
 [65.1 83.7]
 [83.7 83.7]
 [ 9.3 65.1]
 [27.9 65.1]
 [46.5 65.1]
 [65.1 65.1]
 [83.7 65.1]
 [ 9.3 46.5]
 [27.9 46.5]
 [46.5 46.5]
 [65.1 46.5]
 [83.7 46.5]
 [ 9.3 27.9]
 [27.9 27.9]
 [46.5 27.9]
 [65.1 27.9]
 [83.7 27.9]
 [ 9.3  9.3]
 [27.9  9.3]
 [46.5  9.3]
 [65.1  9.3]
 [83.7  9.3]]


In [11]:
test_path = '/mnt/Omer/Project/07.DeepElectrodeMapping/kato/processed_data/20190603_1/phie/'

In [12]:
z_dist_list = [2, 4, 6, 8, 10]
gnd_pos = np.array([[-93, 46, 10]], np.float32)

In [14]:
filesProc.getPecgWithRandomElec(os.path.join(test_path, 'phie'), test_path, elec_pos, z_dist_list, 8, gnd_pos, (93, 93))

In [16]:
pecg = np.load(os.path.join(test_path, 'phie_0000.npy'))
print(pecg)

[[ 0.00000000e+00 -2.51091152e-04 -3.78704047e-04 ...  6.05367587e-01
   6.06267276e-01  6.06929389e-01]
 [ 9.08138645e-04  8.60632076e-04  9.58453385e-04 ...  6.07448994e-01
   6.08208536e-01  6.08715965e-01]
 [ 3.22261965e-03  3.42470962e-03  3.45087209e-03 ...  6.11551575e-01
   6.12442086e-01  6.12789220e-01]
 ...
 [ 5.48730361e-01  5.45345302e-01  5.41430599e-01 ...  2.55648074e-01
   2.54619653e-01  2.53791230e-01]
 [ 5.34944526e-01  5.33041521e-01  5.28516285e-01 ...  2.50062453e-01
   2.49015877e-01  2.48733359e-01]
 [ 5.28790568e-01  5.26756376e-01  5.23099195e-01 ...  2.47066408e-01
   2.46595752e-01  2.46913282e-01]]


In [ ]:
pecg_32 = dataProc.resizeSequence(pecg, (32, 32), cv.INTER_LINEAR)
pecg_32 = dataProc.channelNormalize(pecg_32)
pecg_blocks = dataProc.sequenceToBlocks(pecg_32, 320, 16)

In [ ]:
netg = resUnet3d.getModel(320, 32, 32, 1, 4, 32, 16, 'sigmoid')
netg.load_weights(os.path.join(model_dir, 'netg.h5'))

In [ ]:
pred_blocks = netg.predict(pecg_blocks)
pred  =dataProc.blocksToSequence(pred_blocks, 16)

In [ ]:
video = visualize.makeMovie(pred, 'plasma', 10)
video.save('/mnt/Omer/Project/07.DeepElectrodeMapping/prediction/20190710_1/sim/00_00_050_05000.mp4')